In [1]:
!nvidia-smi

Wed Jan 26 19:37:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 匯入函式庫

In [2]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm


# 匯入資料與前處理

In [3]:
diff_log_data = pd.read_csv('./rolling_linear_residue.csv', index_col=0)
diff_log_data = diff_log_data.astype('float64')
diff_log_data.index = pd.to_datetime(diff_log_data.index)

diff_log_data

,cad_rolling_linear_residue,aud_rolling_linear_residue,gbp_rolling_linear_residue
1981-01-02,-0.000221,-0.000959,-0.002864
1981-01-05,0.005142,0.003863,0.014132
1981-01-06,0.000374,0.003681,0.002493
1981-01-07,0.000255,0.000058,-0.004580
1981-01-08,0.000018,-0.002047,-0.003102
...,...,...,...
2020-12-25,-0.000126,0.001068,-0.000024
2020-12-28,0.001416,-0.003410,-0.008017
2020-12-29,0.002184,0.003571,0.004029
2020-12-30,0.005244,0.010477,0.008856


# CPU/GPU、自定義資料集、模型、訓練函數

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
def SetSeed(myseed):
    # Python random module
    random.seed(myseed)
    # Numpy
    np.random.seed(myseed)
    # Torch
    torch.manual_seed(myseed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(myseed)
        torch.cuda.manual_seed_all(myseed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [6]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, WindowSize):
        X = np.expand_dims(X, 1)
        self.X = X.astype(np.float64)
        self.WindowSize = WindowSize
        
    def __len__(self):
        return len(self.X) - self.WindowSize

    def __getitem__(self, idx):
        return (self.X[idx:idx+self.WindowSize], self.X[idx+self.WindowSize])
        # return (X = [seqs, features], y)

In [7]:
class LSTM(nn.Module):
    def __init__(self, num_layers, hidden_size):
        super().__init__()
        
        self.Input_HiddenLayer = nn.LSTM(input_size=1, hidden_size=hidden_size, num_layers=num_layers)

        self.OutputLayer = nn.Linear(hidden_size, 1)

    def forward(self, input):
        # input.shape = [BatchSize, WindowSize, 1]
        input = input.permute(1, 0, 2)
        # input.shape = [WindowSize, BatchSize, 1]
        hidden, _ = self.Input_HiddenLayer(input)
        # hidden.shape = [WindowSize, BatchSize, HiddenSize]
        hidden = hidden[-1]
        # hidden.shape = [BatchSize, HiddenSize]
        output = self.OutputLayer(hidden)
        
        return output

In [8]:
def train_under_config_and_evaluating_at_num_epochs_list(forex_data,
                                                         length_input_sequence,
                                                         num_epochs_list,
                                                         num_hidden_layers,
                                                         num_hidden_sizes,
                                                         batch_sizes,
                                                         device):
    '''
    forex_data,
    length_input_sequence,
    num_epochs_list,
    learning_rate,
    num_hidden_layers,
    num_hidden_sizes,
    batch_sizes,
    device
    '''
    # setseed
    SetSeed(9527)
    
    # dataset_train
    training_data = forex_data.loc['1981-01-01':'2008-12-31']
    training_dataset = TimeSeriesDataset(training_data, length_input_sequence)
    # dataset_valid
    validation_start_index = len(forex_data.loc['1981-01-01':'2008-12-31']) - length_input_sequence
    validation_end_index = len(forex_data.loc['1981-01-01':'2016-12-31'])
    validation_data = forex_data[validation_start_index:validation_end_index]
    validation_dataset = TimeSeriesDataset(validation_data, length_input_sequence)
    
    # dataloader_train
    training_dataloader = DataLoader(training_dataset, batch_size=batch_sizes, shuffle=True)
    # dataloader_valid
    validation_dataloader = DataLoader(validation_dataset, batch_size=batch_sizes, shuffle=False)
    
    # model
    model = LSTM(num_hidden_layers, num_hidden_sizes).double()
    # criterion & optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters())
    
    # training & evaluating
    min_valid_loss_at_best_epoch = 100000
    min_valid_loss_epoch = 0
    for epoch in tqdm(range(num_epochs_list[-1])):
        # training
        model.to(device)
        model.train()
        for X, y in training_dataloader:
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            ypred = model(X)
            loss = criterion(ypred, y)
            loss.backward()
            optimizer.step()
            
        # evaluating
        if (epoch + 1) in num_epochs_list:
            model.eval()
            valid_loss = 0
            len_valid = 0
            for X, y in  validation_dataloader:
                len_valid += len(X)
                X, y = X.to(device), y.to(device)
                with torch.no_grad():
                    ypred = model(X)
                    loss = criterion(ypred, y)
                valid_loss += loss.detach().cpu().item() * len(X)
            valid_loss = valid_loss / len_valid
            if valid_loss < min_valid_loss_at_best_epoch:
                min_valid_loss_at_best_epoch = valid_loss
                min_valid_loss_epoch = epoch + 1
    
    return min_valid_loss_epoch, min_valid_loss_at_best_epoch

In [9]:
def find_optimum_config_under_specific_input_length(forex_data,
                                                    length_input_sequence,
                                                    num_epochs_list,
                                                    num_hidden_layers_list,
                                                    num_hidden_sizes_list,
                                                    batch_sizes_list,
                                                    device=device):
    min_valid_loss = 100000
    min_valid_config_under_specific_input_length = ()
    for num_hidden_layers in num_hidden_layers_list:
        for num_hidden_sizes in num_hidden_sizes_list:
            for batch_sizes in batch_sizes_list:
                print('\nTraining under config:', (num_hidden_layers, num_hidden_sizes, batch_sizes))
                min_valid_loss_epoch, min_valid_loss_at_best_epoch = train_under_config_and_evaluating_at_num_epochs_list(forex_data,
                                                                                                                          length_input_sequence,
                                                                                                                          num_epochs_list,
                                                                                                                          num_hidden_layers,
                                                                                                                          num_hidden_sizes,
                                                                                                                          batch_sizes,
                                                                                                                          device)
                if min_valid_loss_at_best_epoch < min_valid_loss:
                    min_valid_loss = min_valid_loss_at_best_epoch
                    min_valid_config_under_specific_input_length = (num_hidden_layers, num_hidden_sizes, batch_sizes, min_valid_loss_epoch)

                    print('\nvalid_loss improve to',
                            min_valid_loss,
                            'under config:',
                            min_valid_config_under_specific_input_length,
                            '(num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)')

    print('\nmin valid loss config under specific input length',
          length_input_sequence,
          'is:',
          min_valid_config_under_specific_input_length,
          '(num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)',
          'and the valid loss is:',
          min_valid_loss)
    return min_valid_config_under_specific_input_length, min_valid_loss

# cad

In [ ]:
cad_data = diff_log_data.iloc[:,0]
print(cad_data.loc['1981-01-01':'2008-12-31'])
print(cad_data.loc['2009-01-01':'2016-12-31'])
print(cad_data.loc['2017-01-01':'2020-12-31'])

1981-01-02   -0.000221
1981-01-05    0.005142
1981-01-06    0.000374
1981-01-07    0.000255
1981-01-08    0.000018
                ...   
2008-12-25   -0.001197
2008-12-26   -0.004123
2008-12-29   -0.000227
2008-12-30    0.003186
2008-12-31   -0.002053
Name: cad_rolling_linear_residue, Length: 7304, dtype: float64
2009-01-01   -0.000714
2009-01-02    0.008280
2009-01-05    0.017649
2009-01-06    0.004052
2009-01-07   -0.008306
                ...   
2016-12-26    0.002181
2016-12-27   -0.004858
2016-12-28    0.001239
2016-12-29    0.004212
2016-12-30    0.004732
Name: cad_rolling_linear_residue, Length: 2087, dtype: float64
2017-01-02   -0.000251
2017-01-03    0.001227
2017-01-04    0.009506
2017-01-05    0.005720
2017-01-06   -0.000910
                ...   
2020-12-25   -0.000126
2020-12-28    0.001416
2020-12-29    0.002184
2020-12-30    0.005244
2020-12-31    0.001403
Name: cad_rolling_linear_residue, Length: 1044, dtype: float64


In [ ]:
# random walk mse(validation)
se = cad_data.loc['2009-01-01':'2016-12-31'] ** 2
mse = sum(se) / len(se)
mse

3.594535629597425e-05

In [ ]:
# configuration on length_input_sequence = 5
length_input_sequence = 5
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [ ]:
find_optimum_config_under_specific_input_length(cad_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:33<00:00,  1.11s/it]



valid_loss improve to 3.5989745797743506e-05 under config: (1, 25, 16, 20) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:13<00:00,  2.25it/s]



Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [02:55<00:00,  5.86s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [02:26<00:00,  4.88s/it]


min valid loss config under specific input length 5 is: (2, 1600, 16, 5) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 3.5893125132940946e-05


((2, 1600, 16, 5), 3.5893125132940946e-05)

In [ ]:
# configuration on length_input_sequence = 10
length_input_sequence = 10
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [ ]:
find_optimum_config_under_specific_input_length(cad_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:26<00:00,  1.14it/s]



valid_loss improve to 3.603017564947473e-05 under config: (1, 25, 16, 10) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:14<00:00,  2.14it/s]



valid_loss improve to 3.5991762869471805e-05 under config: (1, 25, 32, 15) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [04:53<00:00,  9.79s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [04:24<00:00,  8.81s/it]


min valid loss config under specific input length 10 is: (3, 1600, 16, 30) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 3.591192145120511e-05


((3, 1600, 16, 30), 3.591192145120511e-05)

In [ ]:
# configuration on length_input_sequence = 20
length_input_sequence = 20
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [ ]:
find_optimum_config_under_specific_input_length(cad_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:28<00:00,  1.04it/s]



valid_loss improve to 3.637055804226643e-05 under config: (1, 25, 16, 20) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:15<00:00,  1.96it/s]



valid_loss improve to 3.597622517856819e-05 under config: (1, 25, 32, 20) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [08:53<00:00, 17.77s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [08:18<00:00, 16.62s/it]


min valid loss config under specific input length 20 is: (2, 25, 256, 5) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 3.590630172830569e-05


((2, 25, 256, 5), 3.590630172830569e-05)

In [ ]:
# configuration on length_input_sequence = 60
length_input_sequence = 60
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [ ]:
find_optimum_config_under_specific_input_length(cad_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:44<00:00,  1.50s/it]



valid_loss improve to 3.5953876486386236e-05 under config: (1, 25, 16, 5) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:20<00:00,  1.44it/s]



Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [25:05<00:00, 50.18s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [24:15<00:00, 48.51s/it]


min valid loss config under specific input length 60 is: (3, 1600, 32, 10) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 3.59202663390862e-05


((3, 1600, 32, 10), 3.59202663390862e-05)

# aud

In [10]:
aud_data = diff_log_data.iloc[:,1]
print(aud_data.loc['1981-01-01':'2008-12-31'])
print(aud_data.loc['2009-01-01':'2016-12-31'])
print(aud_data.loc['2017-01-01':'2020-12-31'])

1981-01-02   -0.000959
1981-01-05    0.003863
1981-01-06    0.003681
1981-01-07    0.000058
1981-01-08   -0.002047
                ...   
2008-12-25   -0.000674
2008-12-26    0.001521
2008-12-29    0.002103
2008-12-30    0.009210
2008-12-31    0.016125
Name: aud_rolling_linear_residue, Length: 7304, dtype: float64
2009-01-01    0.003893
2009-01-02    0.008246
2009-01-05    0.008877
2009-01-06    0.008109
2009-01-07   -0.014846
                ...   
2016-12-26    0.002285
2016-12-27   -0.000916
2016-12-28   -0.001056
2016-12-29    0.005893
2016-12-30   -0.001467
Name: aud_rolling_linear_residue, Length: 2087, dtype: float64
2017-01-02   -0.003278
2017-01-03    0.004918
2017-01-04    0.008884
2017-01-05    0.007581
2017-01-06   -0.004999
                ...   
2020-12-25    0.001068
2020-12-28   -0.003410
2020-12-29    0.003571
2020-12-30    0.010477
2020-12-31    0.001053
Name: aud_rolling_linear_residue, Length: 1044, dtype: float64


In [ ]:
# random walk mse(validation)
se = aud_data.loc['2009-01-01':'2016-12-31'] ** 2
mse = sum(se) / len(se)
mse

6.228930968484775e-05

In [ ]:
# configuration on length_input_sequence = 5
length_input_sequence = 5
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [ ]:
find_optimum_config_under_specific_input_length(aud_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:24<00:00,  1.24it/s]



valid_loss improve to 6.223131960330042e-05 under config: (1, 25, 16, 10) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:12<00:00,  2.31it/s]



Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [02:57<00:00,  5.91s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [02:28<00:00,  4.94s/it]


min valid loss config under specific input length 5 is: (2, 1600, 256, 25) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 6.220922868007452e-05


((2, 1600, 256, 25), 6.220922868007452e-05)

In [11]:
# configuration on length_input_sequence = 10
length_input_sequence = 10
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [12]:
find_optimum_config_under_specific_input_length(aud_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:34<00:00,  1.16s/it]



valid_loss improve to 6.230447099319729e-05 under config: (1, 25, 16, 25) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:13<00:00,  2.23it/s]



valid_loss improve to 6.227089961202687e-05 under config: (1, 25, 32, 25) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [04:51<00:00,  9.72s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [04:20<00:00,  8.70s/it]


min valid loss config under specific input length 10 is: (1, 1600, 256, 5) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 6.221867558746224e-05


((1, 1600, 256, 5), 6.221867558746224e-05)

In [13]:
# configuration on length_input_sequence = 20
length_input_sequence = 20
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [14]:
find_optimum_config_under_specific_input_length(aud_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:28<00:00,  1.06it/s]



valid_loss improve to 6.229232084353771e-05 under config: (1, 25, 16, 10) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:15<00:00,  1.98it/s]



valid_loss improve to 6.222746148749507e-05 under config: (1, 25, 32, 10) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [08:46<00:00, 17.56s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [08:12<00:00, 16.40s/it]


min valid loss config under specific input length 20 is: (1, 400, 128, 5) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 6.221625040344782e-05


((1, 400, 128, 5), 6.221625040344782e-05)

In [15]:
# configuration on length_input_sequence = 60
length_input_sequence = 60
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [16]:
find_optimum_config_under_specific_input_length(aud_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:37<00:00,  1.26s/it]



valid_loss improve to 6.222607106882973e-05 under config: (1, 25, 16, 15) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:21<00:00,  1.41it/s]



Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [24:20<00:00, 48.70s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [23:35<00:00, 47.17s/it]


min valid loss config under specific input length 60 is: (1, 50, 128, 30) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 6.222235649235685e-05


((1, 50, 128, 30), 6.222235649235685e-05)

# gbp

In [10]:
gbp_data = diff_log_data.iloc[:,2]
print(gbp_data.loc['1981-01-01':'2008-12-31'])
print(gbp_data.loc['2009-01-01':'2016-12-31'])
print(gbp_data.loc['2017-01-01':'2020-12-31'])

1981-01-02   -0.002864
1981-01-05    0.014132
1981-01-06    0.002493
1981-01-07   -0.004580
1981-01-08   -0.003102
                ...   
2008-12-25    0.000663
2008-12-26   -0.010867
2008-12-29   -0.012544
2008-12-30    0.001684
2008-12-31    0.012717
Name: gbp_rolling_linear_residue, Length: 7304, dtype: float64
2009-01-01    0.005450
2009-01-02   -0.009023
2009-01-05    0.010761
2009-01-06    0.014373
2009-01-07    0.011438
                ...   
2016-12-26    0.000071
2016-12-27   -0.000828
2016-12-28   -0.003576
2016-12-29    0.003050
2016-12-30    0.006395
Name: gbp_rolling_linear_residue, Length: 2087, dtype: float64
2017-01-02   -0.005184
2017-01-03   -0.003108
2017-01-04    0.007186
2017-01-05    0.007595
2017-01-06   -0.010878
                ...   
2020-12-25   -0.000024
2020-12-28   -0.008017
2020-12-29    0.004029
2020-12-30    0.008856
2020-12-31    0.002988
Name: gbp_rolling_linear_residue, Length: 1044, dtype: float64


In [18]:
# random walk mse(validation)
se = gbp_data.loc['2009-01-01':'2016-12-31'] ** 2
mse = sum(se) / len(se)
mse

3.661160280198625e-05

In [19]:
# configuration on length_input_sequence = 5
length_input_sequence = 5
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [20]:
find_optimum_config_under_specific_input_length(gbp_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:25<00:00,  1.20it/s]



valid_loss improve to 3.71733095460149e-05 under config: (1, 25, 16, 10) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:13<00:00,  2.28it/s]



valid_loss improve to 3.672988338040398e-05 under config: (1, 25, 32, 5) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [02:54<00:00,  5.83s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [02:25<00:00,  4.84s/it]


min valid loss config under specific input length 5 is: (2, 25, 256, 5) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 3.658064989744475e-05


((2, 25, 256, 5), 3.658064989744475e-05)

In [21]:
# configuration on length_input_sequence = 10
length_input_sequence = 10
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [22]:
find_optimum_config_under_specific_input_length(gbp_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:25<00:00,  1.15it/s]



valid_loss improve to 3.6649562451180365e-05 under config: (1, 25, 16, 15) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:13<00:00,  2.18it/s]



Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [04:51<00:00,  9.72s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [04:20<00:00,  8.70s/it]


min valid loss config under specific input length 10 is: (2, 25, 256, 10) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 3.6594920337885236e-05


((2, 25, 256, 10), 3.6594920337885236e-05)

In [11]:
# configuration on length_input_sequence = 20
length_input_sequence = 20
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [12]:
find_optimum_config_under_specific_input_length(gbp_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:37<00:00,  1.23s/it]



valid_loss improve to 3.6652374664194317e-05 under config: (1, 25, 16, 20) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:15<00:00,  1.94it/s]



valid_loss improve to 3.664843260739571e-05 under config: (1, 25, 32, 20) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [08:55<00:00, 17.86s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [08:21<00:00, 16.72s/it]


min valid loss config under specific input length 20 is: (2, 25, 256, 5) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 3.6597572922415864e-05


((2, 25, 256, 5), 3.6597572922415864e-05)

In [13]:
# configuration on length_input_sequence = 60
length_input_sequence = 60
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [14]:
find_optimum_config_under_specific_input_length(gbp_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:37<00:00,  1.24s/it]



valid_loss improve to 3.6637576760927593e-05 under config: (1, 25, 16, 30) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:21<00:00,  1.42it/s]



Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [24:49<00:00, 49.65s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [24:00<00:00, 48.00s/it]


min valid loss config under specific input length 60 is: (2, 25, 256, 5) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 3.659224494666112e-05


((2, 25, 256, 5), 3.659224494666112e-05)